## WELCOME TO WEEK 6

The Epic Finale Week

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK ❤️❤️❤️

### Please note

There may be changes here from the video as I'm always making updates!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To my Windows PC people - an important announcement</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

In [1]:
# 匯入所需的套件與模組

# 載入 dotenv 套件以讀取 .env 檔案中的環境變數
from dotenv import load_dotenv
# 從 openai-agents 套件匯入 Agent（代理人）、Runner（執行器）、trace（追蹤工具）
from agents import Agent, Runner, trace
# 從 agents.mcp 匯入 MCPServerStdio，這是用於啟動 MCP 伺服器的標準輸入/輸出介面
from agents.mcp import MCPServerStdio 
# 匯入 os 套件以操作作業系統相關功能，例如取得目前目錄等
import os

In [2]:
load_dotenv(override=True)

True

### 在 OpenAI Agents SDK 中使用 MCP

1. 建立一個 Client（客戶端）

2. 讓它啟動一個 server（伺服器）

3. 收集該 server 可用的工具

讓我們來試試上週介紹過的 Fetch mcp-server

### 特別注意 - 如果你已經更新了 OpenAI Agents framework：

OpenAI Agents SDK 最近升級並產生了破壞性變更（感謝 OpenAI 😂）。我尚未將本 repo 的版本更新到最新版。如果你自行升級了套件，可能會遇到缺少位置參數的錯誤。如果遇到這個錯誤，你需要將 `await server.list_tools()` 替換為 `await server.session.list_tools()`，並將最後一行從 `fetch_tools` 改為 `fetch_tools.tools`。每次遇到這個錯誤都要做這個修改！這就是在尖端技術上工作的樂趣..

In [3]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get the actual H

## Extra installation step - if you don't have Node and Playwright on your computer

The next MCP tool uses node (the Javascript Server), and it needs you to have the command 'npx' installed on your computer.

Extra setup steps are here:

[Node and Playwright setup](../setup/SETUP-node.md)

## And now 2 more MCP servers - this time Javascript based, with node

### If the next cell hangs, see the important troubleshooting at the end of the SETUP-node instructions linked above.

In [4]:

playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


[Tool(name='browser_close', description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True)),
 Tool(name='browser_resize', description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True)),
 Tool(name='browser_console_messages', description='Returns all console messages', inputSchema={'type': 'object', 'properties': {}, 'additi

In [ ]:

# 取得 sandbox 目錄的絕對路徑，作為檔案系統 MCP server 的根目錄
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
# 設定啟動 MCP 檔案系統 server 的參數，使用 npx 執行 @modelcontextprotocol/server-filesystem，並指定 sandbox 路徑
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

# 使用 MCPServerStdio 啟動檔案系統 server，設定逾時 60 秒
async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    # 取得該 server 可用的工具清單
    file_tools = await server.list_tools()

# 輸出取得的工具清單
file_tools

[Tool(name='read_file', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=None),
 Tool(name='read_text_file', description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates on the file as text regardless of extension. Only work

### And now.. bring on the Agent with Tools!

In [6]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""



async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            print(result.final_output)


I have summarized a great Banoffee Pie recipe in markdown format. However, I am unable to save the file directly due to directory restrictions. Here is the recipe summary you can copy to a file named banoffee.md:

```markdown
# Banoffee Pie Recipe

An easy family favorite with buttery pastry, sweet dulce de leche, bananas, and cream. Serves 8-10.

## Ingredients
- 4 bananas, sliced
- 394g caramel or dulce de leche
- 300ml double cream
- Dark chocolate (optional)

### For the pastry
- 100g butter, chilled (plus extra for greasing)
- 200g plain flour
- 1 medium egg, separated
- 1 tbsp golden caster sugar

## Method
1. Make the pastry case: Put butter and flour in a food processor and pulse until it resembles breadcrumbs. Add egg yolk and sugar, pulse to mix, then add cold water tbsp by tbsp until dough comes together.
2. Remove dough, knead gently to a smooth ball, wrap in cling film and chill for 30 mins.
3. Heat oven to 190C/170C fan/gas 4. Grease a 23cm loose-bottomed fluted tart tin.

### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



